In [1]:
import unittest 
from Beacon_AutoTest import *
from pandas.util.testing import assert_frame_equal
from Test import *
import itertools
import HtmlTestRunner
import time

In [26]:
"""
unit test list
1. extract param range str to dict(model_name,sim_model_info)
2. param_verify(error_flag,test_dataframe,model_dataframe)
3. usecase_verify(error_flag,test_dataframe,usecase_dataframe)
4. reformat_usecase_list(usecase_array)
5. convert_usecase_to_signal_list(usecase_dataframe,usecaselist)
6. append_test_sequence_to_df(test_info_dataframe,params_dict,fine_tune,model,signal_no_list)
7. create_simulink_input_df(test_dataframe, usecase_dataframe)
8. output_excel_create(test_info_dataframe, loglist_dataframe, log_filename)
9. create_matlab_input_list(test_info_dataframe, datalib_dataframe, model_dataframe)
"""

'\nunit test list\n1. extract param range str to dict(model_name,sim_model_info)\n2. param_verify(error_flag,test_dataframe,model_dataframe)\n3. usecase_verify(error_flag,test_dataframe,usecase_dataframe)\n4. reformat_usecase_list(usecase_array)\n5. convert_usecase_to_signal_list(usecase_dataframe,usecaselist)\n6. append_test_sequence_to_df(test_info_dataframe,params_dict,fine_tune,model,signal_no_list)\n7. create_simulink_input_df(test_dataframe, usecase_dataframe)\n8. output_excel_create(test_info_dataframe, loglist_dataframe, log_filename)\n9. create_matlab_input_list(test_info_dataframe, datalib_dataframe, model_dataframe)\n'

In [1]:
class Test(unittest.TestCase):

    @classmethod
    def setUpClass(self):

        self.SimModel = pd.read_excel("Test_SimModel.xlsx",sheet_name="Model")        
        self.Default_Param = pd.read_excel("Test_SimModel.xlsx",sheet_name="Default_Parameter")
        self.SimModel_2 = pd.read_excel("Test_Param.xlsx",sheet_name="Model")
        self.Default_Param_2 = pd.read_excel("Test_Param.xlsx",sheet_name="Default_Parameter")
        self.UseCase = pd.read_excel("RX_Data_Library.xlsx",sheet_name="TestUseCase")
        self.DataLib = pd.read_excel("RX_Data_Library.xlsx",sheet_name="DataLibrary")
    
    ## Set up initialization each time before testing 
    def setUp(self):
        pass
        
            
    ## Tear down some operation each after finish testing
    def tearDown(self):
        pass

    def test_extract_param_range(self):
        
        ## testcase 1
        self.assertEqual(extract_param_range_str_to_dict("MP_test.slx",self.SimModel),{"SAMPLE_TIME:":[0.0,12.0],
                                                                        "Threshold:":[-float("inf"),float("inf")]})
        ## testcase 2
        with self.assertRaises(ValueError) as cm:
            extract_param_range_str_to_dict("MP_test_v2.slx",self.SimModel)
        self.assertEqual(ValueError,cm.expected)
    
        ## testcase 3
        with self.assertRaises(ValueError) as cm:
            extract_param_range_str_to_dict("MP_test_v3.slx",self.SimModel)
        self.assertEqual(ValueError,cm.expected)
   
        ## testcase 4
        ## bug
        self.assertEqual(extract_param_range_str_to_dict("MP_test_v4.slx",self.SimModel),{"SAMPLE_TIME:":[0.0,float("inf")],
                                                                        "WINDOW:":[0.0,float("inf")]})

        with self.assertRaises(IndexError) as cm:
            extract_param_range_str_to_dict("MP_test_v5.slx",self.SimModel)
        self.assertEqual(IndexError,cm.expected)
    
    def test_reformat_usecase_list(self):
        
        ## testcase 1
        self.assertEqual(reformat_usecase_list("Lib1: (1,2,3);Lib2: (4,5)"),"1,2,3,4,5")
        
        ## testcase 2
        self.assertRaises(ValueError,reformat_usecase_list,"Lib1: (1,2,3);")
        
        ## testcase 3
        self.assertRaises(ValueError,reformat_usecase_list,"Lib1:(1,2,3)")
        
        ## testcase 4
        self.assertRaises(ValueError,reformat_usecase_list,"Lib1: (1,2,3),Lib2: (5)")
        
        
    def test_param_verify(self):
        
        ## param checklist - 
        self.assertEqual(param_verify(0,self.Default_Param_2.iloc[[0,1]],self.SimModel_2),0)
        ## bug 
        self.assertEqual(param_verify(0,self.Default_Param_2.iloc[[2]],self.SimModel_2),1)
        self.assertEqual(param_verify(0,self.Default_Param_2.iloc[[3]],self.SimModel_2),1)
        self.assertEqual(param_verify(0,self.Default_Param_2.iloc[[6]],self.SimModel_2),1)
        self.assertEqual(param_verify(0,self.Default_Param_2.iloc[[7]],self.SimModel_2),1)
        self.assertEqual(param_verify(0,self.Default_Param_2.iloc[[5,8]],self.SimModel_2),1)
        
    
    def test_usecase_verify(self):
        
        self.assertRaises(ValueError,usecase_verify,0,self.Default_Param_2.iloc[[4]],self.UseCase)
        self.assertEqual(usecase_verify(0,self.Default_Param_2.iloc[[5]],self.UseCase),1)
        self.assertEqual(usecase_verify(0,self.Default_Param_2.iloc[[1,3]],self.UseCase),1)
        
        
    def test_convert_usecase_to_signal_list(self):
        
        self.assertEqual(convert_usecase_to_signal_list(self.UseCase,str(self.Default_Param_2.iloc[0]["UseCases"]).split(",")),[115,123,179])
        self.assertEqual(convert_usecase_to_signal_list(self.UseCase,str(self.Default_Param_2.iloc[1]["UseCases"]).split(",")),[115,123,179,3324,3334,3344])
        
    
    def test_append_test_sequence_to_df(self):
        

        test_info_dataframe =  pd.DataFrame({'Model_Parameters': None, 'Model_No': None, 'SignalSet': None}, index=range(0, 1))
        expect_dataframe =  pd.DataFrame({'Model_Parameters': None, 'Model_No': None, 'SignalSet': None}, index=range(0, 1))
        params_dict = {"SAMPLE_TIME:": 1,"Threshold:": [1,2]}
        fine_tune = "Threshold:"
        _model = "MP_test.slx"
        signal_no_list = [115,123,179]
        params_dict_list = []
        for item in params_dict[fine_tune]:
            temp = ""
            for _key in params_dict:
                if _key==fine_tune:
                    temp+=(str(_key[:-1])+": "+str(item)+",")
                else:
                    temp+=(str(_key[:-1])+": "+str(params_dict[_key])+",")
            temp = temp[:-1]
            params_dict_list.append(temp)   
        
        comb = [params_dict_list,[_model],signal_no_list]        
        data = list(itertools.product(*comb))
        expect_dataframe= pd.DataFrame(data,columns=['Model_Parameters','Model_No','SignalSet'],index=list(range(1,len(data)+1,1)))        

        test_info_dataframe = append_test_sequence_to_df(test_info_dataframe,params_dict,fine_tune,_model,signal_no_list)
        test_info_dataframe.reset_index(drop=True,inplace=True)
        test_info_dataframe = test_info_dataframe.drop([0])
        test_info_dataframe["SignalSet"] = test_info_dataframe["SignalSet"].astype('int64')

        assert_frame_equal(test_info_dataframe,expect_dataframe)
        
        
        
    
    def test_create_simulink_input_df(self):
        
        test_info_dataframe = pd.DataFrame({'Model_Parameters': None, 'Model_No': None, 'SignalSet': None}, index=range(0, 1))
        expect_dataframe =  pd.DataFrame({'Model_Parameters': None, 'Model_No': None, 'SignalSet': None}, index=range(0, 1))
        params_dict = {"SAMPLE_TIME:": 1.0,"Threshold:": [1.0,2.0]}
        fine_tune = "Threshold:"
        _model = "MP_test.slx"
        signal_no_list = [115,123,179]
        params_dict_list = []
        for item in params_dict[fine_tune]:
            temp = ""
            for _key in params_dict:
                if _key==fine_tune:
                    temp+=(str(_key[:-1])+": "+str(item)+",")
                else:
                    temp+=(str(_key[:-1])+": "+str(params_dict[_key])+",")
            temp = temp[:-1]
            params_dict_list.append(temp)   
        
        comb = [params_dict_list,[_model],signal_no_list]        
        data = list(itertools.product(*comb))
        expect_dataframe= pd.DataFrame(data,columns=['Model_Parameters','Model_No','SignalSet'],
                                       index=list(range(1,len(data)+1,1)))        

        
        ## create dataframe for testbench
        test_info_dataframe = create_simulink_input_df(self.Default_Param_2.iloc[[9]],self.UseCase)
        test_info_dataframe["SignalSet"] = test_info_dataframe["SignalSet"].astype('int64')
        assert_frame_equal(test_info_dataframe,expect_dataframe)
    
    def test_create_matlab_input_list(self):
        
        current_time = time.strftime("%Y-%m-%d-%H-%M") 
        foldername = "'"+"Log_"+current_time+"'"
        test_info_dataframe = pd.DataFrame({'Model_Parameters': None, 'Model_No': None, 'SignalSet': None}, index=range(0, 1))
        entry_testcase = [["'Depth_2cm_Location_0mm_Angle_45.mat'","'SAMPLE_TIME: 1.0,Threshold: 1.0'",
                            "'0'","'MP_test.slx'",foldername,"'/home/ai1/beacon/Beacon_Datalibrary/RX_WithUltrasoundMachines/TissueTest/SonoSite/PGE2a605_0.6mm_day1'",
                            "'/media/ai1/data/beacon/bosheng/algorithmSL'","'MP_test_init.m'","'115'"]]
        test_info_dataframe = create_simulink_input_df(self.Default_Param_2.iloc[[9]],self.UseCase)
        print(test_info_dataframe)
        entry_output = create_matlab_input_list(test_info_dataframe.iloc[[0]],self.DataLib,self.SimModel_2)
        self.assertEqual(entry_output,entry_testcase)
        
           

NameError: name 'unittest' is not defined

In [117]:
if __name__ == "__main__":
    ## when you run this in Jupyter Notebook
    unittest.main(argv=['first-arg-is-ignored'], exit=False,testRunner=HtmlTestRunner.HTMLTestRunner(output='C:\\Users\\900309\\ToDo 6-3-2019'))
    
    ## when you run in IDE
#     unittest.main()


Running tests... 
----------------------------------------------------------------------
 test_append_test_sequence_to_df (__main__.Test) ... OK (0.016006)s
 test_convert_usecase_to_signal_list (__main__.Test) ... OK (0.002002)s
 test_create_matlab_input_list (__main__.Test) ... OK (0.012957)s
 test_create_simulink_input_df (__main__.Test) ... OK (0.013000)s
 test_extract_param_range (__main__.Test) ... FAIL (0.003001)s
 test_param_verify (__main__.Test) ... FAIL (0.002999)s
 test_reformat_usecase_list (__main__.Test) ... OK (0.000000)s
 test_usecase_verify (__main__.Test) ... OK (0.001002)s

FAIL [0.003001s]: __main__.Test.test_extract_param_range
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-116-0f95c4103490>", line 40, in test_extract_param_range
    "WINDOW:":[0.0,float("inf")]})
AssertionError: {'SAMPLE_TIME:': [inf, 0.0], 'WINDOW:': [0.0, inf]} != {'SAMPLE_TIME:': [0.0, inf], 'WINDOW:': [0.0, inf]

In [66]:
a = {"Sam":1,"Tom":[1,2],"Sol":[1,2,3]}

In [67]:
key,value = zip(*a.items()) ##equal to unzip

In [55]:
for i in itertools.product(*value):
    print(dict(zip(key,i)))

{'Sam': 1, 'Tom': 1}
{'Sam': 1, 'Tom': 2}


In [62]:
dict(zip(key,value))

{'Sam': [1], 'Tom': [1, 2], 'Sol': [1, 2, 3]}

In [69]:
value.map()

AttributeError: 'tuple' object has no attribute 'map'

In [60]:
*a.items()

dict_items([('Sam', [1]), ('Tom', [1, 2]), ('Sol', [1, 2, 3])])

In [64]:
import inspect
lins = inspect.getsource(zip)

TypeError: <module 'builtins' (built-in)> is a built-in class